# Objective: generate video from text description

- Download dataset UCF-101 https://www.crcv.ucf.edu/data/UCF101.php. It's an action recognition dataset, with input videos and corresponding action classes.

- Download MoCoGAN code (conditional GAN model for video generation, with categorical condition): https://github.com/DLHacks/mocogan

- Train MoCoGAN on UCF-101: the resulting model will be able to generate videos from action classes

- Download S2VT pre-trained model (video-to-text model): https://vsubhashini.github.io/s2vt.html

  - Note: S2VT uses the Caffe library. Install the library (you don't need to train the model).

- Process each video in UCF-101 and get the corresponding text description. Create a dataset with input=text description and output=action class.

- Train an LSTM classifier (similar to the one used in class for sentiment analysis) to classify text descriptions into actions. Report the performance.

- The final model works by: getting an input text description by the user, converting it into an action class with the LSTM model, and using the action class as a condition to MoCoGAN.

Note: the model will not be precise in generating video details, since conditioning is based on the class only. 
For example:

"I'm running in the park" -> action: running

"My dog is running on the beach" -> action: running

Output videos will not actually take into account the context, but only the action.

In [2]:
import os
import torch
from glob import glob
from torch.utils.data import DataLoader

In [9]:
import skvideo.io

ModuleNotFoundError: No module named 'skvideo'

In [8]:
data_dir = "/home/carlo/Documents/Cognitive Computing/Text2VideoGAN"
image_dir_name = "UCF-101"
image_paths = glob(os.path.join(data_dir, image_dir_name, "*"))

if (len(image_paths) < 1):
    data_dir = "/home/carlo/Documenti/Text2VideoGAN/"
    image_paths = glob(os.path.join(data_dir, image_dir_name, "*"))
    
image_paths

['/home/carlo/Documenti/Text2VideoGAN/UCF-101/Haircut',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/Basketball',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/SalsaSpin',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/Bowling',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/BrushingTeeth',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/Skijet',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/PlayingGuitar',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/RockClimbingIndoor',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/SumoWrestling',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/Surfing',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/WritingOnBoard',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/Biking',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/PoleVault',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/BodyWeightSquats',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/SkyDiving',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/HulaHoop',
 '/home/carlo/Documenti/Text2VideoGAN/UCF-101/Apply